## Hopfield Modell

zunächst werdenen die benötigten Bibliotheken implementiert

In [2]:
import numpy as np
import cv2 #Für Kammera
import tkinter as tk #Für Gui
import matplotlib.pyplot as plt
import tkinter.filedialog
import matplotlib
import threading
import random
matplotlib.use('TkAgg')

In [225]:
def on_key(event):
    print(f'Taste {event.key} gedrückt, schließe alle Fenster')
    plt.close('all')

def plot_params(training_patterns, testing_pattern):
    #Funktion Plottet die Training Mustter und Testing Musster in einer Reihe
    num_training_patterns = len(training_patterns)
    fig1, axs1 = plt.subplots(1, num_training_patterns + 1, figsize=(num_training_patterns * 4, 4))
    
    for i, pattern in enumerate(training_patterns):
        axs1[i].imshow(pattern, cmap='gray')
        axs1[i].set_title(f'Training {i+1}')
        axs1[i].set_xticks([])
        axs1[i].set_yticks([])
    
    axs1[num_training_patterns].imshow(testing_pattern, cmap='gray')
    axs1[num_training_patterns].set_title('Testing Pattern')
    axs1[num_training_patterns].set_xticks([])
    axs1[num_training_patterns].set_yticks([])

    fig1.suptitle('Netzwerkparamter')
    plt.tight_layout()

def capture_image():
    # Open the webcam (0 is the default device)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return None

    # Capture a single frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Error: Could not capture image")
        return None

    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray

def crop_center(image, crop_size):
    #Schneidet einen zentralen Ausschnitt aus einem Bild.
    h, w = image.shape
    startx = w // 2 - (crop_size // 2)
    starty = h // 2 - (crop_size // 2)
    return image[starty:starty + crop_size, startx:startx + crop_size]



def edit_image(gray_image,matrix_size, crop_size=None):
    #Bild wird zugeschnitten
    if crop_size is None:
        crop_size = 700

    if gray_image is not None:
        img = crop_center(gray_image,crop_size)
        img = np.array(cv2.resize(img, (matrix_size,matrix_size))) #Hier wird das bild in ein quadratisches Array umgewandelt
        mean = np.mean(img) # Der Mittelwert aller Grauwerte 0 bis 255 wird genommen.
        img = np.where( img >= mean, 1, -1) # Array einträge werden mit berücksichtugung des Mittelwerts auf 1 und -1 gesetzt mit Heaviside-Funktion
        return img
        
    else:
        print("No image captured.")

def flip_values(matrix, probability):
    #Funktion zum Zufälligen umdrehen zufälliger Matrixdaten
    flipped_matrix = matrix.copy()
    flip_mask = np.random.rand(*matrix.shape) < probability
    flipped_matrix[flip_mask] *= -1
    return flipped_matrix

def flip_matrix_entries(matrix, p):
    # Erstelle eine Kopie der Eingabematrix
    flipped_matrix = np.copy(matrix)

    # Ermittelt die Gesamtzahl der Einträge in der Matrix.
    total_entries = flipped_matrix.size
    
    # Berechnen Sie die Anzahl der zu drehenden Einträge.
    num_to_flip = int(total_entries * p)
    
    # Ermittelt die Indizes der Matrixeinträge
    indices = np.indices(flipped_matrix.shape).reshape(2, -1).T
    
    # Wählen Sie zufällig die Indizes aus, die umgedreht werden sollen.
    flip_indices = np.random.choice(len(indices), size=num_to_flip, replace=False)
    
    # Spiegeln der ausgewählten Einträge
    for idx in flip_indices:
        i, j = indices[idx]
        if flipped_matrix[i, j] == 1:
            flipped_matrix[i, j] = -1
        elif flipped_matrix[i, j] == -1:
            flipped_matrix[i, j] = 1
            
    return flipped_matrix

def read_images_from_folder(folder_path):
    #Funktion um Bilder aus dem Bilder-Ordner zu etrahieren.
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:
                images.append(img)
            else:
                print(f"Could not read image: {img_path}")
    return images


### UI zum wählen von Mustern

In [226]:
class PixelGridApp:
    def __init__(self, master, mode):
        self.master = master
        self.master.title("Hopfield Modell Musterauswahl")
        self.grid_size = 5
        self.cell_size = 40 # Jede Zelle ist 40 X 40 Pixel Groß
        self.patterns = []
        self.mode = mode # Modus zur bestimmung ob man im testing oder im trainig modus ist.

        self.grid = np.full((self.grid_size, self.grid_size), 1, dtype=int)
        self.canvases = []

        self.create_widgets()
        self.update_mode_label()
    
    # Im folgenden werden die ganzen Bausteinen und Knöppfe definiert. Den Knöpfen wird jeweils eine Funktion zugeteil die weiter unten definiert sind.
    def create_widgets(self):
        for i in range(self.grid_size):
            row_canvases = []
            for j in range(self.grid_size):
                canvas = tk.Canvas(self.master, width=self.cell_size, height=self.cell_size, bg="white")
                canvas.grid(row=i, column=j)
                canvas.bind("<Button-1>", lambda event, i=i, j=j: self.toggle_pixel(i, j))
                canvas.create_text(self.cell_size//2, self.cell_size//2, fill="black", font=("Helvetica", 16))
                row_canvases.append(canvas)
            self.canvases.append(row_canvases)

        self.save_button = tk.Button(self.master, text="Save Pattern", command=self.save_pattern)
        self.save_button.grid(row=self.grid_size, column=0, columnspan=self.grid_size // 2)

        self.reset_button = tk.Button(self.master, text="Reset Grid", command=self.reset_grid)
        self.reset_button.grid(row=self.grid_size, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.save_file_button = tk.Button(self.master, text="Save to File", command=self.prompt_save_file)
        self.save_file_button.grid(row=self.grid_size + 1, column=0, columnspan=self.grid_size // 2)

        self.load_file_button = tk.Button(self.master, text="Load from File", command=self.prompt_load_file)
        self.load_file_button.grid(row=self.grid_size + 1, column=self.grid_size // 2, columnspan=self.grid_size // 2)

        self.finish_button = tk.Button(self.master, text="Finish", command=self.finish)
        self.finish_button.grid(row=self.grid_size + 2, column=0, columnspan=self.grid_size)

        self.mode_label = tk.Label(self.master, text="")
        self.mode_label.grid(row=self.grid_size + 3, column=0, columnspan=self.grid_size)
    
    #Ändert die verschiedenen Modi
    def update_mode_label(self):
        if self.mode == "training":
            self.mode_label.config(text="Mode: Training")
        elif self.mode == "testing":
            self.mode_label.config(text="Mode: Testing")

    #Ändert die Pixel von schwartz zu weiß und umgekehrt und ordnet diesen die werte 1 bzw. -1 zu
    def toggle_pixel(self, i, j):
        if self.grid[i, j] == 1:
            self.grid[i, j] = -1
            self.canvases[i][j].config(bg="black")
            self.canvases[i][j].itemconfig(1, fill="white")
        else:
            self.grid[i, j] = 1
            self.canvases[i][j].config(bg="white")
            self.canvases[i][j].itemconfig(1, fill="black")
    
    # Eingetipptes Pattern wird in einem Array gespeichert, dieses hängt vom Modus ab
    def save_pattern(self):
        self.patterns.append(self.grid.copy())
        print("Pattern saved:")
        print(self.grid)
        if self.mode == 'testing':
            self.testing_mode()
    
    #Alle Pixel werden auf weiß gesetzt
    def reset_grid(self):
        self.grid.fill(1)
        for row_canvases in self.canvases:
            for canvas in row_canvases:
                canvas.config(bg="white")
                canvas.itemconfig(1, fill="black")
        print("Grid reset")
    
    # Die beieden untern funktionen dienen zur speicherung der Arrays als txt datei und zum automatischen ändern des Moduses im nachhinein
    def prompt_save_file(self):
        filename = tk.filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.save_patterns_to_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
    
    def save_patterns_to_file(self, filename):
        np.savetxt(filename, np.array(self.patterns).reshape(len(self.patterns), -1), fmt='%d')
        print(f"Patterns saved to {filename}")
    
    #Ähnliche funktion wie zum speichern, nur das hier txt. Datein geladen werdne können
    def prompt_load_file(self):
        filename = tk.filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("All files", "*.*")])
        if filename:
            self.load_patterns_from_file(filename)
            if self.mode == "training":
                self.switch_to_testing_mode()
            if self.mode == "testing":
                self.testing_mode()
            
    
    def load_patterns_from_file(self, filename):
        data = np.loadtxt(filename, dtype=int)
        if self.mode == "testing":
            self.patterns = data.reshape(1, self.grid_size, self.grid_size).tolist()
        else:
            num_patterns = data.shape[0]
            self.patterns = data.reshape(num_patterns, self.grid_size, self.grid_size).tolist()
        print(f"Patterns loaded from {filename}")
    
    #Hier wird aktiv zum testingmodus gewächselt und das Training-Speicherpattern wird abgespeichert
    def switch_to_testing_mode(self):
        self.training_patterns = np.array(self.patterns)
        self.mode = "testing"
        self.patterns = []
        self.update_mode_label()
        self.reset_grid()
        print("Switched to Testing mode")

    #Hier wird das Testing Pattern im Zugehörigen Array gespeichert
    def testing_mode(self):
        self.testing_pattern = np.array(self.patterns)
        print('Testing Pattern saved in Array')

    #Funktion beendet das Programm
    def finish(self):
        try:
            self.testing_pattern
        except:
            self.testing_pattern = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')
        try:
            self.training_patterns
        except:
            self.training_patterns = np.zeros((5, 5))
            print('Error: Nichtsts gespeichert')   
        self.master.quit()  # Beendet die Tkinter-Mainloop
        self.master.destroy()  # Zerstört das Tkinter-Fenster


Jetzt wird zunächst das Hopfieldmodel implementiert

In [244]:
class HopfieldNetwork:
    
    #Initierungsfunktion
    def __init__(self, num_neurons, steps):
        self.num_neurons = num_neurons
        self.matrix_size = int(np.sqrt(self.num_neurons))
        self.weights = np.zeros((num_neurons, num_neurons))
        self.steps = steps
        self.plot_nth_time = 1  # Standardwert für plot_nth_time
        self.patterns = []

    #Trainig durch die Hebbsche Regel. Normiert auf die Anzahl der Neuronen. Input sind gegebene Patterns
    def train(self, patterns):
        self.patterns = patterns
        for p in patterns:
            p = p.flatten()  # Ensuring pattern is a column vector
            if len(p) != self.num_neurons:
                raise ValueError(f"Pattern size {len(p)} does not match the number of neurons {self.num_neurons}.")
            self.weights += np.outer(p, p)
        self.weights /= self.num_neurons
        np.fill_diagonal(self.weights, 0)  # No self-connections
        
    #Da hoft mit den Pattern als Vektoren statt als Matritzen gearbeitet wird, müssen diese nach den Rechenoperationen wieder in Matritzen_Form zu verfügng stehen
    def show_as_matrix(self,pattern):
        matrix = np.reshape(pattern, (self.matrix_size, self.matrix_size))
        return matrix
    
    # Hier wird versucht eine Vorschleife zu erstellen, die die Temperaturinverse betta abhänging vom Ordnungsparameter minimiert.
    #Funktioniert jedoch noch nicht ganz, da Start Mustter immer als optimal audgegeben wird
    def predict_beta(self, pattern, activation_function, init_beta, trainig_steps):
        s = pattern.flatten().copy()
        m_0_max = self.m_biggest(self.patterns, s)
        m_arr_max = [m_0_max]
        beta =init_beta
        l = len(range(self.steps))
        beta_step = init_beta/l
        beta =init_beta
        betas = [init_beta]
        max_m_t =[]
        for i in range(trainig_steps):
            for i in range(self.steps):
                a = np.dot(self.weights, s)
                s = activation_function(a, beta)
                m_max = self.m_biggest(self.patterns, s)
                m_arr_max.append(m_max)
                s = s.flatten()
            betas.append(beta)
            beta = beta-beta_step
            m_end = m_arr_max[self.steps-1]
            max_m_t.append(m_end)
        m_max_index = np.argmax(np.abs(max_m_t))
        # Element mit dem größten absoluten Wert auswählen
        m_max_end = max_m_t[m_max_index]
        b_crit = betas[m_max_index]
        return  b_crit, m_max_end  


    # Diese Funktion ist der Kern des Hopfield Models. hier wird die Update und aktivierungsregel angewendet.
    #Zusätzlich wird der aktuelle Gesamtenrgie-Wert berechnent und in einem Array gespeichert.
    #Genauso der Order Parameter. Dieser wird einmal für alle Trainigspattern gemittelt.
    #Und es wird einmal nur der größte Ordnungsparameter verwendet, der dan tendenziel von dem Trainingspattern stammt,  zu dem konvergeirt werden soll.
    def predict(self, pattern, activation_function, axs=None, beta=None, plot_nth_time=None):
        if plot_nth_time is None:
            plot_nth_time = 1
        s = pattern.flatten().copy()  # Kopie des Musters und Umformung zu Spaltenvektor
        eng_array = np.array([self.energy(s)])
        m_0 = self.m_mean(self.patterns, s)
        m_0_max = self.m_biggest(self.patterns, s)
        m_array = [m_0]
        m_arr_max = [m_0_max]
        plot_index = 0
        for i in range(self.steps): # Hier wird das Ergebnis als neues Testing pattern erneut auf die Gewichtsmatrix angewendet. 
            a = np.dot(self.weights, s)
            s = activation_function(a, beta)
            s = s.flatten()
            eng = self.energy(s)
            m = self.m_mean(self.patterns, s)
            m_max = self.m_biggest(self.patterns, s)
            m_array.append(m)
            m_arr_max.append(m_max)
            eng_array = np.append(eng_array, eng)
            
            if axs is not None and i % plot_nth_time == 0: # Die Pattern nach N Steps werden hier direkt geplottet.
                plot_index = int(i/plot_nth_time)
                axs[plot_index].imshow(self.show_as_matrix(s), cmap='gray')
                axs[plot_index].set_title(f'Nach {i+1} Steps')
                axs[plot_index].set_xticks([])
                axs[plot_index].set_yticks([])
                plt.tight_layout()
        #Hier werden die 3 aufgenommenen Parametern als instanzen der Klasse abgespeichert, um später auf sie zukreifen zu können.
        self.eng_array = eng_array
        self.m_array = m_array
        self.m_array_max = m_arr_max
        return s.flatten()  # Rückgabe als flaches Array

    # Funktionen zur berechnung des Order Parameters und der Energie
    #Ordnungsparameter für ein einziges Pattern
    def m_singel(self, pattern, state):
        pattern = pattern.flatten().copy()
        state = state.flatten().copy()
        return np.dot(pattern, state) /self.num_neurons # Formel zur berechnung des Ordnungsparameters
    
    #GewichteterOrdnungsparameter
    def m_mean(self, patterns, state):
        m = [self.m_singel(pattern, state) for pattern in patterns]
        m_mean = np.mean(m)
        return m_mean
    #Nur der akluell größte Ordnungsparameter wird wieder gegeben.
    def m_biggest(self, patterns, state):
        m = [self.m_singel(pattern, state) for pattern in patterns]
        # Index des Elements mit dem größten absoluten Wert finden
        m_max_index = np.argmax(np.abs(m))
        # Element mit dem größten absoluten Wert auswählen
        m_max = m[m_max_index]
        return m_max

    #Gesamenergie für einen bestimmten Zustand
    def energy(self, pattern):
       
        return -0.5 * np.dot(np.dot(pattern.T, self.weights), pattern)
    

#Aktivierungsfunktionen
#Hier werden die Verschiedenen Aktivierungsfunktionen definiert.

#Für Kontinuirliches Modell
def tanh(a):
    return np.tanh(a)

#Heaviside Funktion
def heviside(a, g=None):
    return np.where(a >= 0, 1, -1)

# Stochastisches Aktivierungsfunktion für einzelne Werte
def stochastic(a, beta):
    def prop(a, beta):
        return 1/(1+np.exp(-2*beta*a))
    return 1 if random.random() < prop(a, beta) else -1

# Stochastische Aktivierungsfunktion für Arrays
def stochastic2(a, beta):
    def prop(a, beta):
        return 1/(1+np.exp(-2*beta*a))
    
    # Sicherstellen, dass die Funktion ein Array mit der gleichen Form wie "a" zurückgibt
    prob_array = prop(a, beta)
    
    # Zufallswerte für den Vergleich generieren
    random_values = np.random.rand(*a.shape)
    
    # Elementweiser Vergleich durchführen
    return np.where(random_values < prob_array, 1, -1)# Nur Zufällige werte die den Prob array genügen werden verwendet, so wird dje wahrscheinlichkeit eingeführt
    




### Energiefunktion für 2 Neuronen Netzwerk

In [263]:
# Inputs bzw. Zustände
x1 = np.linspace(-2,2,100)
x2 = np.linspace(-2,2,100)
x_1, x_2 = np.meshgrid(x1,x2)

# Gewicht
w= 1

# Energiefunktion für 2 Spin Ising Kette oder 2 Neuron Hopfield Modell
def E(x1,x2,w):
    return -0.5*w*x1*x2

#Plotting
z = E(x_1,x_2,w)
b=1
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(np.tanh(b*x_1), np.tanh(b*x_2), z, cmap='viridis')

# Achsenbeschriftungen
ax.set_xlabel(r'$x_1$ Input')
ax.set_ylabel(r'$x_2$ Input')
ax.set_zlabel('Energie')

# Zeige den Plot
plt.show()

### Programm mit Frontkamera

In [273]:
#images
#Hopfield Network starten mit gewünschten Parameter
network = HopfieldNetwork(num_neurons= 10000, steps= 4)
steps = network.steps 
matrix_size = network.matrix_size  
#Bildaxe und Plot initalisieren
fig1, axs1 = plt.subplots(1, 2, figsize=(3 * 4, 4))
fig2, axs2 = plt.subplots(1, steps, figsize=(3 * 4, 4))

# Bild Aufnehmen und lernen
gray_image = capture_image()
learn_image = edit_image(gray_image, matrix_size)
axs1[0].imshow(learn_image, cmap='gray')
axs1[0].set_title('Aufgenommenes Bild')
network.train([learn_image])

#Rauschen hinzufügen

noise_level = 0.1
noise_img = flip_values(learn_image,noise_level)

axs1[1].imshow(noise_img, cmap='gray')
axs1[1].set_title(f'Bild mit Einem Noiselevel von {noise_level*100}% ')

fig1.suptitle('Netzwerkparamter')

#Netzwerk Anwenden

network.predict(noise_img,heviside, axs=axs2)
fig2.suptitle('Netzwerkausgabe')
plt.show()
#print(network.weights)



### Programm mit Pixelauswahl

In [268]:
#UI wird gestartet
plt.close('all')
root = tk.Tk()
app = PixelGridApp(root, "training")
#und beendet
root.mainloop()

#Wir herhalten hieraus dei beiden Arrays für das Training und das Testing
training = app.training_patterns
testing = app.testing_pattern[0]

# Hopfieldmodell wird initiallisiert
network = HopfieldNetwork(num_neurons=25, steps=7)
steps = network.steps

#Netzwerk wird mit den Trainingsdaten gefüttert
network.train(training)
plot_params(training_patterns=training, testing_pattern=testing)

#Figure für die Step Bilder werden hier erstellt
fig2, axs2 = plt.subplots(1, steps, figsize=(len(training) * 4, 4))

#Dem Netzwerk werden die Trainingsdaten gegeben und es wird Laufen gelassen
network.predict(testing,heviside, axs=axs2)

#Hier folgt nur noch das Plotting
fig2.suptitle('Netzwerkausgabe')
fig3, axs3 = plt.subplots(1,1)
axs4 = axs3.twinx()
line1,=axs4.plot(np.arange(steps+1), network.m_array_max, color ='forestgreen', label = r'm$_{max}$')
line2,=axs4.plot(np.arange(steps+1), network.m_array, color ='lightgreen', label = r'm$_{mean}$')
axs4.set_ylabel('Ordnungsparamter')
axs3.grid()
fig3.suptitle('Energiefunktion und Ordnungsparameter')
axs3.set_xlabel('Schritte')
axs3.set_ylabel('Energie')
line3, = axs3.plot(np.arange(steps+1),network.eng_array, label = 'Energie')
# Erstellen von Legenden
lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
axs3.legend(lines, labels)
plt.show()

#print(network.m_array)


Pattern saved:
[[ 1  1 -1  1  1]
 [ 1 -1  1 -1  1]
 [ 1 -1 -1 -1  1]
 [-1 -1  1 -1 -1]
 [-1  1  1  1 -1]]
Grid reset
Pattern saved:
[[ 1  1 -1  1  1]
 [ 1  1 -1  1  1]
 [ 1  1 -1  1  1]
 [ 1  1 -1  1  1]
 [ 1 -1 -1  1  1]]
Grid reset
Pattern saved:
[[ 1 -1 -1  1 -1]
 [-1  1  1 -1  1]
 [-1  1 -1  1  1]
 [ 1 -1  1  1  1]
 [-1  1  1  1  1]]
Grid reset
Pattern saved:
[[ 1  1  1  1  1]
 [ 1 -1 -1 -1  1]
 [ 1 -1  1 -1  1]
 [ 1 -1 -1 -1  1]
 [ 1  1  1  1  1]]
Patterns saved to /Users/phileasmeier/Documents/UNI/Neurale_Netzwerke/Hopfield_Model/Hopefield_Model/daniels_training.txt
Grid reset
Switched to Testing mode
Testing Pattern saved in Array
Pattern saved:
[[ 1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1]
 [ 1  1 -1  1 -1]
 [ 1 -1  1  1 -1]
 [-1  1  1  1  1]]
Testing Pattern saved in Array
Pattern saved:
[[ 1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1]
 [ 1  1 -1  1 -1]
 [ 1 -1  1  1 -1]
 [-1  1  1  1  1]]
Testing Pattern saved in Array


Stochastic

In [271]:
plt.close('all')
import os
import cv2
import numpy as np

network = HopfieldNetwork(num_neurons= 2500, steps= 30)
steps = network.steps 
beta_init = 10
plot_times = 7
t = 0.4
testing_steps = 30


    
# Bilder werden vom Ordner geladen und in arrays umgewandelt
folder_path = 'Bilder'
images = read_images_from_folder(folder_path)
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]
gray_images = [edit_image(img,network.matrix_size, 700) for img in gray_images]

#Trining Daten sind die Bilder aus dem Ordner
training = gray_images   

#Das Netzwerk wird Trainirt
network.train(training)
#beta_crit, m_max = network.predict_beta(testing, stochastic2, init_beta= beta_init, trainig_steps= testing_steps)
#print(beta_crit, m_max)

#Belibiges Bild wird verauscht
testing = flip_matrix_entries(training[3],t)
plot_params(training,testing)
fig2, axs2 = plt.subplots(1, int(np.floor((steps/plot_times)))+1,  figsize=(int(np.floor((steps/plot_times)))*4, 4))

#Netzwwerk werden die Testadaten gegeben und es wird ausgeführt
network.predict(testing,stochastic2, axs=axs2, beta=beta_init , plot_nth_time=plot_times)

# Überprüfen Sie die Anzahl der geladenen Bilder
print(f"Number of images read: {len(images)}")

#Ergebnisse werden geplottet
fig3, axs3 = plt.subplots(1,1)
axs3.grid()
fig3.suptitle('Energiefunktion Und Ordnungsparameter')
axs3.set_xlabel('Schritte')
axs3.set_ylabel('Energie')
#eng_x = np.delete(np.arange(steps+1),0)
eng_x = np.arange(steps+1)
#eng_y = np.delete(network.eng_array,0)
eng_y = network.eng_array
line3, axs3.plot(eng_x,eng_y)

axs4 = axs3.twinx()
line1, =axs4.plot(np.arange(steps+1), network.m_array_max, color ='forestgreen', label = r'm$_{max}$')
line2, =axs4.plot(np.arange(steps+1), network.m_array, color ='lightgreen', label = r'm$_{mean}$')
axs4.set_ylabel('order parameter')
fig3.canvas.mpl_connect('key_press_event', on_key)
fig2.canvas.mpl_connect('key_press_event', on_key)
#fig1.canvas.mpl_connect('key_press_event', on_key)

lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
axs3.legend(lines, labels)
plt.show()



Number of images read: 9


### Ab hier Plots für die Präsentation

In [223]:

plt.figure(figsize=(4,4))
plt.xlim(-2,2)
plt.ylim(-2,2)
# Das Achsen-Objekt des Diagramms in einer Variablen ablegen:
ax = plt.gca()

# Die obere und rechte Achse unsichtbar machen:
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Die linke Diagrammachse auf den Bezugspunkt '0' der x-Achse legen:
#ax.spines['left'].set_position(('data',0))

# Die untere Diagrammachse auf den Bezugspunkt '0' der y-Achse legen:
#ax.spines['bottom'].set_position(('data',0))

# Ausrichtung der Achsen-Beschriftung festlegen:
#ax.xaxis.set_ticks_position('bottom')
#ax.yaxis.set_ticks_position('left')
plt.xticks([-1,0,1,2])
plt.yticks([-1,0,1,2])
ax.plot([1,-1],[1,-1], 'o', color = 'skyblue')
ax.plot([1,-1],[-1,1], 'o')
ax.set_xlabel('$x_1$', fontsize ='14')
ax.set_ylabel('$x_2$', fontsize ='14')
ax.grid()
# Achse-Beschriftungen durch weiß-transparenten Hintergrund hervorheben:
plt.tight_layout()
plt.savefig('prs1',dpi=400)
